In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager

## NASA Mars News

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\kyucz\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# URL of page to be scraped
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

# Create Beautiful Soup object & select parser
html = browser.html
soup = bs(html, 'lxml')

# Extract title text
article = soup.find('div', class_="list_text")
news_title = article.find('div', class_="content_title").text

# Extract paragraph text
news_p = article.find('div', class_="article_teaser_body").text

# Print attributes
print(news_title)
print(news_p)

NASA Ingenuity Mars Helicopter Prepares for First Flight
Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  


In [4]:
# exit browser
browser.quit()

## JPL Mars Space Images - Featured Image

In [5]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\kyucz\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [6]:
images_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html#'
browser.visit(images_url)

# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

# Use splinter to navigate the site & find the image URL for the current featured mars image
# assign url string to variable: featured_image_url
find_img = soup.find('img', class_="headerimage fade-in")['src']

# make sure to find the image url to the full size .jpg image
# save complete url string for the image
base_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
featured_image_url = base_url + find_img
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg


In [7]:
# exit browser
browser.quit()

## Mars Facts

In [8]:
# define url of site to be scraped
facts_url = 'https://space-facts.com/mars/'

# use read_html function in pandas to scrape all tabular data
tables = pd.read_html(facts_url)
tables

# slice off 1st dataframe
marsfacts = tables[0]
marsfacts.columns=['Category','Mars Profile']
marsfacts.reset_index(drop=True, inplace=True)

# generate html table from dataframe
mars_html_table = marsfacts.to_html(index=False)

# save directly to html file
marsfacts.to_html('mars_facts_table.html')

In [9]:
# strip unwanted newlines to clean up table
# mars_html_table = mars_html_table.replace('\n', '')
# mars_html_table

## Mars Hemispheres

In [16]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\kyucz\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [17]:
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemi_url)

# establish base url variable to append links to for each image in loop
base_url = 'https://astrogeology.usgs.gov'

# Create empty list to store title & url info into a dictionary for each hemisphere
mars_hemis = []

# Iterate through all images
#for i in range(0, 4):
# HTML object
html = browser.html

#Parse HTML with beautiful soup
soup = bs(html, 'html.parser')

# Retrieve all elements that contain image info
results = soup.find('div', class_='result-list')
images = soup.find_all('div', class_='description')

for image in images:
    # Use BS's find() method to navigate main page to retrieve titles
    title = image.find('h3').text
    title = title.replace("Enhanced","")
    link = image.find('a')['href']
    hemi_link = base_url + link
    # Visit each page to retrieve full jpg url
    browser.visit(hemi_link)
    html = browser.html
    soup = bs(html, 'html.parser')
    wide_image = soup.find('img', class_='wide-image')
    img_url = wide_image['src']
    full_img_url = base_url + img_url
    mars_hemis.append({'Title': title, 'img_url': full_img_url})

    #browser.back()
    #browser.links.find_by_partial_href('enhanced').click()

In [18]:
mars_hemis

[{'Title': 'Cerberus Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere ',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [19]:
browser.quit()